In [ ]:
import numpy as np
import xarray as xr
from pathlib import Path
import re
import geopandas as gp

from glacier_flow_tools.profiles import extract_profile_simple

In [ ]:
profile_resolution = 250
profiles_path = Path("../glacier_flow_tools/data/greenland-flux-gates-29.gpkg")
profiles_gp = gp.read_file(profiles_path).rename(columns={"id": "profile_id", "name": "profile_name"})
geom = profiles_gp.segmentize(profile_resolution)
profiles_gp = gp.GeoDataFrame(profiles_gp, geometry=geom)
profiles_gp = profiles_gp[["profile_id", "profile_name", "geometry"]]


In [ ]:
regexp = "v(.+?)"
dim = "exp_id"

In [ ]:

for k, profile in profiles_gp.iterrows():
    if k == 0:
        bm_files = Path("data/").glob("BedMachine*.nc")
        extracted_profiles = []
        for k, p in enumerate(sorted(list(bm_files))):
            ds = xr.open_dataset(p, chunks="auto")
            ds.expand_dims(dim)
            m_id_re = re.search(regexp, ds.encoding["source"])
            m_id = int(m_id_re.group(1))
            ds[dim] = m_id
            extracted_profiles.append(extract_profile_simple(profile, ds))
    all_profiles = xr.concat(extracted_profiles, dim="exp_id")

In [ ]:
all_profiles.profiles.plot(title="Test")

In [ ]:
657675-657575